In [1]:
# Only for google Colab
!pip install cebra
from google.colab import drive
drive.mount('/content/drive/')
!unzip /content/drive/MyDrive/Cebra_data/2020_11_9_MV1_run_brain.zip


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 3.5 MB/s eta 0:00:00
Mounted at /content/drive/
Archive:  /content/drive/MyDrive/Cebra_data/2020_11_9_MV1_run_brain.zip
   creating: 2020_11_9_MV1_run_brain/
  inflating: 2020_11_9_MV1_run_brain/nomove_0001_0001.tif  
  inflating: 2020_11_9_MV1_run_brain/nomove_0001_0002.tif  
  inflating: 2020_11_9_MV1_run_brain/nomove_0001_0003.tif  
  inflating: 2020_11_9_MV1_run_brain/nomove_0001_0004.tif  
  inflating: 2020_11_9_MV1_run_brain/nomove_0001_0005.tif  
  inflating: 2020_11_9_MV1_run_brain/move_0001_0001.tif  
  inflating: 2020_11_9_MV1_run_brain/move_0001_0002.tif  
  inflating: 2020_11_9_MV1_run_brain/move_0001_0003.tif  
  inflating: 2020_11_9_MV1_run_brain/move_0001_0004.tif  
  inflating: 2020_11_9_MV1_run_brain/move_0001_0005.tif  
  inflating: 2020_11_9_MV1_run_brain/move_0002_0001.tif  
  inflating: 2020_11_9_MV1_run_brai

In [2]:
!unzip /content/drive/MyDrive/Cebra_data/2020_11_9_MV1_run_behavior.zip

Archive:  /content/drive/MyDrive/Cebra_data/2020_11_9_MV1_run_behavior.zip
  inflating: nomove_0060_0002.tif    
  inflating: nomove_0060_0001.tif    
  inflating: nomove_0059_0001.tif    
  inflating: nomove_0058_0002.tif    
  inflating: nomove_0058_0001.tif    
  inflating: nomove_0057_0002.tif    
  inflating: nomove_0057_0001.tif    
  inflating: nomove_0056_0002.tif    
  inflating: nomove_0056_0001.tif    
  inflating: nomove_0055_0006.tif    
  inflating: nomove_0055_0005.tif    
  inflating: nomove_0055_0004.tif    
  inflating: nomove_0055_0003.tif    
  inflating: nomove_0055_0002.tif    
  inflating: nomove_0055_0001.tif    
  inflating: nomove_0054_0003.tif    
  inflating: nomove_0054_0002.tif    
  inflating: nomove_0054_0001.tif    
  inflating: nomove_0053_0003.tif    
  inflating: nomove_0053_0002.tif    
  inflating: nomove_0053_0001.tif    
  inflating: nomove_0052_0003.tif    
  inflating: nomove_0052_0002.tif    
  inflating: nomove_0052_0001.tif    
  inflating: 

In [4]:
!mkdir 2020_11_9_MV1_run_behavior


mkdir: cannot create directory ‘2020_11_9_MV1_run_behavior’: File exists


In [6]:
!mv *.tif 2020_11_9_MV1_run_behavior/

mv: cannot stat '.*.tif': No such file or directory


In [7]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import cebra.datasets
from cebra import CEBRA
from PIL import Image
import cv2
import torch
import pickle

/usr/local/lib/python3.10/dist-packages/cebra/datasets/monkey_reaching.py:35: UserWarning: Could not import the nlb_tools package required for data loading of cebra.datasets.monkey_reaching. Dataset will not be available. If required, you can install the dataset by running pip install git+https://github.com/neurallatents/nlb_tools.
  warnings.warn(


In [8]:
if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"
print(DEVICE)

cuda


In [9]:
from logging import exception
def process_brain(path):
  try:
    brain_seq = cv2.imreadmulti(path)[1]
    brain_seq = np.array(brain_seq)
    brain_mask = (np.sum(brain_seq, axis=0) > 0)
    flat_seq = (brain_seq[:, brain_mask])
    return flat_seq.astype(float)
  except:
    print(np.shape(brain_seq))


In [10]:
# iterate through all brain data and process it
#https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory
import os
neural_data = []
name_data = []
behav_data = []

filepath = "2020_11_9_MV1_run_brain"
for file in os.listdir(filepath):
     filename = os.fsdecode(file)
     if filename.endswith(".tif"):
         neural_data.append(process_brain(filepath + '/' + filename))
         name_data.append(filename)
         split_name = filename.split('_')
         session_data = []
         session_data.append(int(split_name[2].split('.')[0]))
         session_data.append(int(split_name[1]))
         behav_data.append(session_data)
         continue
     else:
         continue

In [11]:
# using https://github.com/facebookresearch/dinov2
# extract feature labels from images
import torch
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vits14_pretrain.pth
100%|██████████| 84.2M/84.2M [00:00<00:00, 232MB/s]


In [24]:
from torchvision import transforms as pth_transforms

preprocess = pth_transforms.Compose([
        pth_transforms.Resize(256, interpolation=3),
        pth_transforms.CenterCrop(224),
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

In [25]:
def get_features(model, image):
  return model(preprocess(image).unsqueeze(0))

In [31]:
  from PIL import Image
  import numpy as np

  def np_to_PIL(numpy_image):
      return Image.fromarray(np.uint8(numpy_image)).convert('RGB')

In [32]:
def process_behav(path):
  beh_seq = cv2.imreadmulti(path)[1]
  beh_seq = np.array(beh_seq)
  feature_seq = []
  for beh_feat in beh_seq:
    feature_seq.append(get_features(dinov2_vits14, np_to_PIL(beh_feat)))
  return np.array(feature_seq)


In [ ]:
behav_feature = []
name_data_behav = []

filepath = "2020_11_9_MV1_run_brain"
for file in os.listdir(filepath):
     filename = os.fsdecode(file)
     if filename.endswith(".tif"):
         behav_feature.append(process_behav(filepath + '/' + filename))
         name_data.append(filename)
         split_name = filename.split('_')
         continue
     else:
         continue

In [ ]:

#generate mock behavioural data for time only run
mock_behav = []
for count, data in enumerate(neural_data):
    mock_behav.append(np.arange(np.size(data[:, 0]), dtype=float))


In [ ]:
out_dim = 8
max_iterations = 10

In [ ]:
multi_cebra_model = cebra.CEBRA(batch_size=1,
                                output_dimension=out_dim,
                                max_iterations=200,
                                verbose=True,
                                max_adapt_iterations=10)

In [ ]:
multi_cebra_model.fit(neural_data[:50], mock_behav[:50])

pos:  0.8668 neg:  3.0214 total:  3.8882 temperature:  1.0000: 100%|██████████| 200/200 [01:06<00:00,  3.00it/s]


CEBRA(batch_size=1, max_adapt_iterations=10, max_iterations=200, verbose=True)

In [ ]:
print(np.shape(mock_behav[1]))
np.shape(neural_data[1])

(101,)


(101, 10634)

In [ ]:


fig = plt.figure()

ax1 = plt.subplot(141, projection='3d')

for run, data in enumerate(neural_data[:50]):
  embedding = multi_cebra_model.transform(data, session_id=run)
  ax1 = cebra.plot_embedding(embedding, ax=ax1)

plt.show()
